In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import gensim
from gensim.models import Word2Vec

from sklearn.preprocessing import LabelEncoder
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Attention, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.preprocessing.text import Tokenizer

In [2]:
# Load your dataset from the CSV file (replace 'your_dataset.csv' with the actual file path)
dataset_path = 'new_work.csv'
df2 = pd.read_csv(dataset_path)
df2


,input_text,aspect,polarity,extracted_nouns
0,decor nice casual fine dining,ambience,conflict,"['dining', 'decor']"
1,seating trying get maximum amount people resta...,ambience,negative,"['people', 'amount', 'neighbor', 'seating']"
2,like took leftover chicken poured oil sprinkle...,food,negative,"['chicken', 'powder', 'sauce', 'red', 'pepper'..."
3,finest latin women youll see life hang jimmys,ambience,positive,"['life', 'women']"
4,ive bunch times service always outstanding,service,positive,"['service', 'times', 'bunch']"
...,...,...,...,...
2028,sure previous reviewer lonk dined saul great n...,food,positive,"['reviewer', 'food', 'neighborhood']"
2029,bruscetta bit soggy salads fresh included nice...,food,conflict,"['mix', 'bit', 'iceberg', 'kitchen', 'dishes',..."
2030,waitstaff attentive polite helpful impressive ...,service,positive,"['feat', 'waitstaff', 'quarters']"
2031,prices beat quality quantity freshness taste,food,positive,"['quantity', 'prices', 'freshness', 'taste', '..."


In [3]:
# unique categories

unique_categories = df2['polarity'].unique()

print("Unique categories in the column:")
for category in unique_categories:
    print(category)

Unique categories in the column:
conflict
negative
positive
neutral


In [4]:
df2.to_csv('new_work.csv', index=False)

In [5]:
# List of terms
sentence = df2['input_text'].tolist()
aspect = df2['aspect'].tolist()
print(sentence)
print(aspect)

['decor nice casual fine dining', 'seating trying get maximum amount people restaurant nice neighbor dine', 'like took leftover chicken poured oil sprinkled pepper powder sauce translucent red', 'finest latin women youll see life hang jimmys', 'ive bunch times service always outstanding', 'food outstanding little perks great', 'makhani ok korma bland', 'make reservations expect delayed 1520 minutes hosting staff difficulty seating guests arrived reservation probably lot walk ins close time square', 'need kick sushi pretty good place consistent', 'keep capex minimum put cash bagels among best city', 'wont break bank also wouldnt come back food', 'staff attentive ambience lovely food superb', 'incredible food warm atmosphere friendly service downtown neighborhood spot doesnt miss beat', 'leon east village gem casual hip well prepared basic french bistro fare good specials warm lively atmosphere', 'one us actually liked expresso thats', 'reservations expensive tips annoying stuff', 'porti

In [6]:
# Pair sentences with their corresponding aspect terms
sentence_aspect_pairs = list(zip(sentence, aspect))

In [7]:
# Print the first few sentence-aspect pairs
for t, a in sentence_aspect_pairs[:5]:
    print(f"Sentence Tokens: {a}\nAspect Term: {a}\n")

Sentence Tokens: ambience
Aspect Term: ambience

Sentence Tokens: ambience
Aspect Term: ambience

Sentence Tokens: food
Aspect Term: food

Sentence Tokens: ambience
Aspect Term: ambience

Sentence Tokens: service
Aspect Term: service



In [8]:
# Tokenize sentences
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
# Combine sentences and aspect terms
filtered_sentence_tokens = [word_tokenize(sentence.lower()) for sentence in sentence]
aspect_tokens = [word_tokenize(item.lower()) for item in aspect]
all_tokens = filtered_sentence_tokens + filtered_sentence_tokens
all_tokens

[['decor', 'nice', 'casual', 'fine', 'dining'],
 ['seating',
  'trying',
  'get',
  'maximum',
  'amount',
  'people',
  'restaurant',
  'nice',
  'neighbor',
  'dine'],
 ['like',
  'took',
  'leftover',
  'chicken',
  'poured',
  'oil',
  'sprinkled',
  'pepper',
  'powder',
  'sauce',
  'translucent',
  'red'],
 ['finest', 'latin', 'women', 'youll', 'see', 'life', 'hang', 'jimmys'],
 ['ive', 'bunch', 'times', 'service', 'always', 'outstanding'],
 ['food', 'outstanding', 'little', 'perks', 'great'],
 ['makhani', 'ok', 'korma', 'bland'],
 ['make',
  'reservations',
  'expect',
  'delayed',
  '1520',
  'minutes',
  'hosting',
  'staff',
  'difficulty',
  'seating',
  'guests',
  'arrived',
  'reservation',
  'probably',
  'lot',
  'walk',
  'ins',
  'close',
  'time',
  'square'],
 ['need', 'kick', 'sushi', 'pretty', 'good', 'place', 'consistent'],
 ['keep',
  'capex',
  'minimum',
  'put',
  'cash',
  'bagels',
  'among',
  'best',
  'city'],
 ['wont', 'break', 'bank', 'also', 'wouldnt

In [13]:
# Initialize the Word2Vec model
model = Word2Vec(
    vector_size=100,  # Adjust dimensionality as needed
    window=5,
    min_count=1,
    workers=4,
    sg=0,  # Use CBOW architecture (skip-gram: sg=1)
)

In [14]:
# Build the vocabulary
model.build_vocab(all_tokens)

In [15]:
# Train the model
model.train(all_tokens, total_examples=model.corpus_count, epochs=10)

(292333, 328240)

In [17]:
# Save the model
model.save("word2vec.model")

print("Word2Vec model trained and saved.")

Word2Vec model trained and saved.


In [18]:
# Load Word2Vec model
model_file = 'word2vec.model'  # Replace with your model file path
word2vec_model = Word2Vec.load(model_file)

In [28]:
# Convert tokenized sentences and aspect terms to embeddings
sentence_embeddings = np.array([np.mean([model.wv[token] for token in sentence], axis=0) for sentence in filtered_sentence_tokens])
category_embeddings = np.array([np.mean([model.wv[token] for token in aspect], axis=0) for aspect in aspect_tokens])

In [29]:
# Assuming you have a DataFrame 'df' with a column 'polarity'
polarity_mapping = {'positive': 0, 'negative': 1, 'neutral': 2, 'conflict': 3}
df2['polarity_numeric'] = df2['polarity'].map(polarity_mapping)

invalid_labels_df = df2[~df2['polarity_numeric'].isin([0, 1, 2, 3])]
print(invalid_labels_df)

Empty DataFrame
Columns: [input_text, aspect, polarity, extracted_nouns, polarity_numeric]
Index: []


In [30]:
# Prepare labels
labels = df2['polarity_numeric'].values
print(labels)
invalid_labels = [item for item in labels if item not in [0, 1, 2, 3]]
print(invalid_labels)

[3 1 1 ... 0 0 0]
[]


In [31]:
from keras.layers import Input, Dense, LSTM, Bidirectional, Concatenate, Attention, Reshape
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.layers import Dropout
from keras.optimizers import RMSprop


In [32]:
# Define model architecture
sentence_input_layer = Input(shape=(sentence_embeddings.shape[1],))  # Input shape based on sentence embeddings
category_input_layer = Input(shape=(category_embeddings.shape[1],))  # Input shape based on category embeddings

# Add a time dimension to the sentence embeddings
sentence_input_layer_reshaped = Reshape((1, sentence_embeddings.shape[1]))(sentence_input_layer)

# Transform category_embeddings to have the same dimension as bi_lstm_layer
transform_layer = Dense(256, activation='relu')  # 128 to match the dimension of bi_lstm_layer
transformed_category_embeddings = transform_layer(category_input_layer)

bi_lstm_layer = Bidirectional(LSTM(128, return_sequences=True))(sentence_input_layer_reshaped)
category_attention = Attention()([bi_lstm_layer, transformed_category_embeddings])

concatenated_layer = Concatenate(axis=-1)([bi_lstm_layer, category_attention])
flattened_layer = Flatten()(concatenated_layer)
# output_layer = Dense(4, activation='softmax')(flattened_layer)
output_layer = Dense(256, activation='relu')(flattened_layer)  # Increase Dense units to 256
output_layer = Dropout(0.5)(output_layer)  # Add dropout
output_layer = Dense(4, activation='softmax')(output_layer)

model = Model(inputs=[sentence_input_layer, category_input_layer], outputs=output_layer)

In [62]:
# Split data into train, validation, and test sets
X_train_sentence, X_temp_sentence, y_train, y_temp = train_test_split(sentence_embeddings, labels, test_size=0.2, random_state=42)
X_val_sentence, X_test_sentence, y_val, y_test = train_test_split(X_temp_sentence, y_temp, test_size=0.5, random_state=42)

X_train_category, X_temp_category = train_test_split(category_embeddings, test_size=0.2, random_state=42)
X_val_category, X_test_category = train_test_split(X_temp_category, test_size=0.5, random_state=42)

In [63]:
# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=4)
y_val = to_categorical(y_val, num_classes=4)

In [64]:
print("Unique labels:", np.unique(y_train))
print("Number of unique labels:", len(np.unique(y_train)))
print("Any missing labels:", np.isnan(y_train).any())

print("Unique labels after one-hot encoding:", np.unique(y_train))
print("Shape of y_train after one-hot encoding:", y_train.shape)
print("Count of samples for each class:", np.sum(y_train, axis=0))

Unique labels: [0. 1.]
Number of unique labels: 2
Any missing labels: False
Unique labels after one-hot encoding: [0. 1.]
Shape of y_train after one-hot encoding: (1626, 4)
Count of samples for each class: [1029.  409.   85.  103.]


In [65]:
# Compile and train the model
optimizer = RMSprop(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit([X_train_sentence, X_train_category], y_train, epochs=100, validation_data=([X_val_sentence, X_val_category], y_val))

Epoch 1/100
51/51 [==============================] - 6s 40ms/step - loss: 0.9869 - accuracy: 0.6181 - val_loss: 0.9858 - val_accuracy: 0.6207
Epoch 2/100
51/51 [==============================] - 1s 22ms/step - loss: 0.9621 - accuracy: 0.6304 - val_loss: 0.9699 - val_accuracy: 0.6207
Epoch 3/100
51/51 [==============================] - 1s 18ms/step - loss: 0.9555 - accuracy: 0.6304 - val_loss: 0.9917 - val_accuracy: 0.6207
Epoch 4/100
51/51 [==============================] - 1s 13ms/step - loss: 0.9436 - accuracy: 0.6335 - val_loss: 0.9690 - val_accuracy: 0.6207
Epoch 5/100
51/51 [==============================] - 1s 13ms/step - loss: 0.9426 - accuracy: 0.6322 - val_loss: 0.9677 - val_accuracy: 0.6207
Epoch 6/100
51/51 [==============================] - 1s 13ms/step - loss: 0.9390 - accuracy: 0.6353 - val_loss: 0.9634 - val_accuracy: 0.6207
Epoch 7/100
51/51 [==============================] - 1s 13ms/step - loss: 0.9405 - accuracy: 0.6384 - val_loss: 0.9539 - val_accuracy: 0.6207
Epoch 

In [67]:
# Save the model to a file
model.save('my_model.h5')

# Later, load the model from the file
from keras.models import load_model
model = load_model('my_model.h5')

In [68]:
from sklearn.metrics import accuracy_score

In [69]:
# Convert labels to one-hot encoding
y_test = to_categorical(y_test, num_classes=4)

In [70]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate([X_test_sentence, X_test_category], y_test)

7/7 [==============================] - 1s 5ms/step - loss: 0.9129 - accuracy: 0.6176


In [71]:
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

Test loss: 0.9129161238670349
Test accuracy: 0.6176470518112183
